# Uso de RAG para recuperar conhecimento organizacional

## Objetivo: Criar uma rag

In [1]:
%%capture --no-stderr
%pip install langchain langchain-community langchain-text-splitters dotenv langchain-google-genai chromadb pypdf langchain-mistralai ipywidgets unstructured

In [2]:
import os, getpass
from dotenv import load_dotenv

load_dotenv()  # Isso carrega automaticamente as variáveis do .env para o os.environ


def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("GROQ_API_KEY")
_set_env("GOOGLE_API_KEY")

In [3]:
from langchain.schema import Document
from langchain.document_loaders import PDFMinerLoader

def carregar_documentos(pdf_path_folder: str) -> list[Document]:
    documentos = []
    for fn in os.listdir(pdf_path_folder):
        if fn.lower().endswith(".pdf"):
            loader = PDFMinerLoader(os.path.join(pdf_path_folder, fn))
            documentos.extend(loader.load())
    return documentos


In [4]:
from langchain.text_splitter import CharacterTextSplitter

def dividir_documentos(documentos: list[Document],
                       chunk_size: int = 1000,
                       chunk_overlap: int = 0) -> list[Document]:
    """
    Divide cada Document em pedaços menores (chunks) para melhorar a qualidade dos embeddings.
    """
    splitter = CharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    documentos_divididos = splitter.split_documents(documentos)
    return documentos_divididos

In [5]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import Chroma

def criar_e_armazenar_embeddings(documentos: list[Document],
                                 persist_directory: str = "chroma_langchain_db",
                                 collection_name: str = "km-db-new") -> Chroma: # Changed collection name
    """
    Gera embeddings usando GoogleGenerativeAIEmbeddings e armazena em um índice Chroma.
    Se desejar persistência em disco, especifique `persist_directory`.
    """
    embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")
    
    store = Chroma.from_documents(
        documentos,
        embeddings,
        persist_directory=persist_directory,
        collection_name=collection_name
    )
    return store

In [6]:
from langchain_google_genai import ChatGoogleGenerativeAI

def configurar_llm():
    return ChatGoogleGenerativeAI(model="gemini-2.0-flash")

In [7]:
from langchain.chains import RetrievalQA

folder = "/home/gui/rag-knowledge-managment/docs/pdfs"

# 2. Carrega e processa documentos
documentos = carregar_documentos(folder)
documentos_divididos = dividir_documentos(documentos)

# 3. Cria embeddings e armazena em Chroma
vetor_store = criar_e_armazenar_embeddings(documentos_divididos)

# 4. Configura o LLM para consultas
llm = configurar_llm()

# 5. Consulta RAG usando RetrievalQA
rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",             # opções: "stuff", "map_reduce", "refine"
    retriever=vetor_store.as_retriever(k=3),
    return_source_documents=True
)

# Sua pergunta (prompt)
pergunta = "Qual é o papel do agente de Product Ops em um time de desenvolvimento ágil?"

# Roda a consulta
result = rag_chain.invoke(pergunta)

# Exibe resultados
print("Resposta:\n", result["result"])
print("\nDocumentos usados:")
for doc in result["source_documents"]:
    fonte = doc.metadata.get("source", "<sem fonte>")
    print(f"- {fonte}")


Created a chunk of size 1262, which is longer than the specified 1000
Created a chunk of size 1071, which is longer than the specified 1000
Created a chunk of size 1667, which is longer than the specified 1000
Created a chunk of size 1134, which is longer than the specified 1000
Created a chunk of size 1130, which is longer than the specified 1000
Created a chunk of size 1165, which is longer than the specified 1000
Created a chunk of size 1081, which is longer than the specified 1000
Created a chunk of size 1134, which is longer than the specified 1000
Created a chunk of size 1383, which is longer than the specified 1000
Created a chunk of size 1041, which is longer than the specified 1000
Created a chunk of size 1586, which is longer than the specified 1000
Created a chunk of size 1791, which is longer than the specified 1000
Created a chunk of size 1721, which is longer than the specified 1000
Created a chunk of size 1234, which is longer than the specified 1000


Resposta:
 Product Ops desempenha um papel crucial em uma equipe de desenvolvimento ágil, focando em otimizar e acelerar o processo de desenvolvimento do produto. Eles fazem isso através de:

*   **Automatização e Padronização:** Automatizam tarefas repetitivas e padronizam formatos e padrões de qualidade para entradas e saídas das equipes, liberando tempo para trabalho mais significativo.
*   **Identificação e Remoção de Gargalos:** Medem e avaliam continuamente o processo de desenvolvimento, estudam dependências entre equipes e identificam gargalos para acelerar o lançamento de funcionalidades.
*   **Priorização Informada:** Compreendem os custos de P&D, objetivos de negócios e necessidades dos stakeholders, fornecendo sugestões para priorizar o trabalho de desenvolvimento para máximo impacto.
*   **Alinhamento:** Estabelecem cadências regulares para reuniões multifuncionais e regras para consulta e engajamento de equipes.
*   **Medição e Melhoria Contínua:** Medem continuamente o pr